# Webscraping 1: Webscraping and Data Curation

This notebook will be the first step in our exploration of *webscraping*, the process of (semi-)automatically collecting data from the internet. In this lesson, our goal will be to **populate a dataset of classical Persian poetry**. We will scrape this data from [Ganjoor.net]("https://ganjoor.net"), a repository of classical Persian poetry. This material is not well studied by Western scholars, but is a treasure trove of information about the classical Persian literary tradition.

Specifically, we are going to use the Python package `BeautifulSoup`. This tool is designed to parse and traverse the underlying structure of websites, known as Hyper Text Markup Language or HTML, so that we can query it for relevant information. `BeautifulSoup` is not hard to understand, but it does require a good grounding in HTML. To that end, we will need to jump back and forth between Ganjoor and this notebook to understand how to take advantage of `BeautifulSoup`. This is to simulate how many of these webscraping projects go. At the end of the day, they are iterative, often involving the programmer to bouce between the webpage and their code several times. Like all of the skills in this course, this requires some practice to get good at.

Without further delay, let's dive into webscraping!

In [ ]:
from bs4 import BeautifulSoup # BeautifulSoup constructor
import requests # requests allows us to "ask for" the data at the URL
import pandas as pd # we'l be using pandas to store our collected information

## Access the web in Python

Although we have the URL to the site that we want to scrape, it may not be obvious how we can access it using Python. That's where the package `requests` comes in. This library allows us to utilize the core functionality of an internet browser in our Python code. In this case, we'll be using a GET request to get the HTML of a website.

If you are new to the inner workings of the internet, this may seem confusing. Simply put, when you access a webpage on your computer, your browser sends a GET request (the exact same thing we're going to be doing in all of our webscraping exercises) to the URL. The server which is hosting the URL interprets this request and, if we are allowed, will send back all of the information that we need to recreate the website on our own computer. The browser then renders this information in a way that allows it to be human readable.

We are going to do the same thing below, but instead of making it human readable, we will keep the raw code of the site and use it to extract valuable information.

Below we see an example of a GET request.

In [ ]:
url = "https://ganjoor.net"
response = requests.get(url) # ask requests to "GET" the URL
response # 200 means that it worked (as opposed to 404)
# but where's the data?

In [ ]:
response.text # we can get the raw HTML with the `text` attribute

In [ ]:
# what this site actually looks like
from IPython.display import IFrame
IFrame(url, width=800, height=400)

## Using our soup

### Parsing the home page

Now that we have some HTML, the fun can begin. This HTML is what `BeautifulSoup` expects to be given. `BeautifulSoup` allows us then to call up specific parts of the site that we know are important to our goal. This is where you, as the programmer, need to have a very deep understanding of the structure of the website you are trying to scrape.

In this case, the poetry for each poet is stored per poet on the front page. Specifically, when a user clicks on a poet's name, they are brought to a list of their works, each of which then takes them to a given work. Because we are trying to get all of the text of all of the works, we're going to need to collect these URLs so that we can get the data from each of them. The first step is to get the URLs for each poet page. We'll see how to do that below.

In [ ]:
soup = BeautifulSoup(response.text, "html.parser") # constructing the soup of the home page

In [ ]:
print(soup.prettify()) # printing the soup, compare to the visual above

From some exploration, I know that all of the poet URLs are held in `div` tags with a class attribute equal to `poet`. We can use `BeautifulSoup` to get all of these in a list.

In [ ]:
poets = soup.find_all("div", attrs={"class": "poet"})
len(poets)

In [ ]:
for i, p in enumerate(poets):
    print(i, p.find("a")["href"])

If you pay close attention to the list above you can see that the first six authors are repeated later and that the last link is to a map. We'd like to exclude these first six, as they are a bit different than the others and we'll alo want to exlude the map, as its content is very different. To that end, we'll use index slicing to restrict our results.

Importantly, we need to do this because of a pecularity of *this* website. Most sites will have something like this, where most links follow a general pattern, but that some components need to be manually excluded. So you will need to keep a close eye on corner cases that could throw off your code.

In [ ]:
poet_urls = [url+p.find("a")["href"] for p in poets[6:-1]] # limiting to just the author pages
df = pd.DataFrame(poet_urls, columns=["poet_url"]) # load the urls into a dataframe
df

### Parsing poet pages

Now that we have a `DataFrame` of all of the poet pages, we can take each one and pull out the URLs to each of their works. We will take each URL, submit a GET request and then parse the content as a new soup object.

In [ ]:
df.poet_url.iloc[0]

In [ ]:
roodaki_response = requests.get(df.poet_url.iloc[0])
roodaki_soup = BeautifulSoup(roodaki_response.text, "html.parser")
print(roodaki_soup.prettify())

Just like the home page, all of the relevant links are kept in `div` tags with class attribute `part-title-block`. Importantly, I only knew this because I inspected this HTML very closely.

In [ ]:
roodaki_soup.find_all("div", attrs={"class":"part-title-block"})

In [ ]:
# need the hrefs from these
poem_tags = roodaki_soup.find_all("div", attrs={"class":"part-title-block"})
poem_tags[0]

In [ ]:
poem_tags[0].find('a') # finds just the FIRST instance of the tag
# equal to this
poem_tags[0].find_all('a')[0]
# also equal to this
poem_tags[0].a

In [ ]:
# html attr work like dictionaries
poem_tags[0].a['href']

In [ ]:
# can use a loop to get all of the links
poem_links = []
for p in poem_tags:
    poem_links.append(url+p.a['href'])

# this is equal to
poem_links = [url+p.a['href'] for p in poem_tags]
poem_links

In [ ]:
# can turn this into a function so we can use it for all of the urls
def get_poet_urls(p_url):
  response = requests.get(p_url)
  soup = BeautifulSoup(response.text, "html.parser")
  return [url+d.a['href'] for d in soup.find_all("div", class_="part-title-block")]

In [ ]:
url_list = get_poet_urls(df.poet_url.iloc[0])
url_list

In [ ]:
# can take advantage of the df
df['poem_url'] = df.poet_url.apply(get_poet_urls) # takes (a lot of) time, why?
df.poem_url.iloc[0]

In [ ]:
# our function returns a list
# but we'd like a unique url to be in a unique row
poem_df = df.explode('poem_url').dropna() # explode separates lists into individual rows
poem_df

### Getting the text of each poem

Just like how we got links from the home page, we were able to get the links to individual poems from the poet pages. Now we have a long list of all of the poem links, but unfortunately, we still need to go one level further: each one of these poem pages contains a list of links to poem excerpts which have the actual text we're interested. We will follow the same pattern as above.

In [ ]:
poem_df.poem_url.iloc[0]

In [ ]:
baghimande_response = requests.get(poem_df.poem_url.iloc[0])
baghimande_soup = BeautifulSoup(baghimande_response.text, "html.parser")
print(baghimande_soup.prettify())

In [ ]:
# all of the excerpts have the class "poem-excerpt"
baghimande_soup.find_all("p", attrs={"class":"poem-excerpt"})

In [ ]:
# as we saw above
for p in baghimande_soup.find_all("p", attrs={"class":"poem-excerpt"}):
    print(p.a['href'])

In [ ]:
def get_poem_links(p_url):
  response = requests.get(p_url)
  soup = BeautifulSoup(response.text, "html.parser")
  return [url+d.a['href'] for d in soup.find_all("p", attrs={"class":"poem-excerpt"})]

In [ ]:
get_poem_links(poem_df.poem_url.iloc[0])

In [ ]:
poem_df

In [ ]:
poem_df['excerpt_url'] = poem_df.poem_url.apply(get_poem_links) # takes (a lot of) time, why?
poem_df

In [ ]:
excerpt_df = poem_df.explode('excerpt_url').dropna()
excerpt_df

### Collecting the data

We are now finally ready to start collecting the text of each poem excerpt. Unfortunately, as I'm sure you noticed in the last cells, this code is starting to take a long time. It takes about 1 second to send a GET request, to parse it with `BeautifulSoup` and then get all of the tags we need. This doesn't seem like a long time in isolation, but above we needed to process 512 links, meaning that it took about 512 seconds, which is about eight and a half minutes. Without more advanced software, there's nothing we can do about this.

To that end, I'll show an example of scraping the text from one of these pages for just one full poem, but be ware there are many, many more. When I ran this on my laptop in full, it look about 6 hours to get all of the text. Thankfully, I saved the data after, meaning we never need to do that again!

In [ ]:
ex_excerpt = excerpt_df.excerpt_url.iloc[0]
ex_excerpt

In [ ]:
ex_excerpt_response = requests.get(ex_excerpt)
ex_excerpt_soup = BeautifulSoup(ex_excerpt_response.text, "html.parser")
lines = soup.find_all('div', attrs={'class':'b'}) # lines are stored in div tags with class b

In [ ]:
for line in lines:
    print(line.text)

In [ ]:
def get_poem_text(p_url):
    response = requests.get(p_url)
    soup = BeautifulSoup(response.text, "html.parser")
    lines = soup.find_all('div', attrs={'class':'b'})
    return [line.text for line in lines]

In [ ]:
# dont run, it will take a long time
# excerpt_df['text'] = excerpt_df.poem_links.apply(get_poem_text)
# text_df = excerpt_df.explode('text').dropna()

In [ ]:
# instead here's the file i ran myself
!wget https://tufts.box.com/shared/static/a4i21wis54oh4xxwwvi2chrll7sq7gvb.csv -O full_ganjoor_poems.csv
ganjoor = pd.read_csv('full_ganjoor_poems.csv')
ganjoor

In [ ]:
# poems of hafez
hafez = ganjoor[ganjoor.poet == 'hafez']
hafez.poem.unique()

In [ ]:
full_ghazal = '\n'.join(ganjoor[ganjoor.poem == 'ghazal'].poem_lines.dropna().to_list())
full_ghazal[:500]